In [1]:
import json
import pandas as pd  

f = open('human_data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

human_df = pd.DataFrame(data)
f = open('ai.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
ai_df = pd.DataFrame(data)
ai_df.columns = ['text', 'label']
df_concat = pd.concat([human_df, ai_df])

# Shuffle the concatenated dataframe
df_shuffled = df_concat.sample(frac=1).reset_index(drop=True)

# Print the shuffled dataframe
#print(df_shuffled)
df = df_shuffled

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer
  

nltk.download('stopwords')
nltk.download('punkt')

# Define Turkish stopwords
stop_words = set(stopwords.words('turkish'))

# Define Turkish stemmer
#stemmer = SnowballStemmer('turkish')
stemmer=TurkishStemmer()

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text.lower())

    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Stem the remaining words
    tokens = [stemmer.stemWord(token) for token in tokens]

    # Rejoin the words into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bahadir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/bahadir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from nltk.tokenize import word_tokenize
max_words = 200
df['text'] = df['text'].apply(preprocess_text)
df['text'] = df['text'].apply(lambda x: ' '.join(word_tokenize(x)[:max_words]))



In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, stratify = df['label'])

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6527 entries, 3248 to 8069
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6527 non-null   object
 1   label   6527 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 153.0+ KB


In [6]:
data = []
for index, row in train.iterrows():
    label = '__label__' + str(row['label'])
    text = row['text']
    example = label + ' ' + text
    data.append(example)

# Save the examples to a text file
with open('fasttext_train_data.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')
        
data = []
for index, row in test.iterrows():
    label = '__label__' + str(row['label'])
    text = row['text']
    example = label + ' ' + text
    data.append(example)

# Save the examples to a text file
with open('fasttext_test_data.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')

In [7]:
import fasttext
# load the training data
train_data = "fasttext_train_data.txt"
# load the testing data
test_data = "fasttext_test_data.txt"

# train the model
model = fasttext.train_supervised(input=train_data, epoch=25,lr=1.0)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])



Read 1M words
Number of words:  61221
Number of labels: 2
Progress:  93.9% words/sec/thread: 4420114 lr:  0.060847 avg.loss:  0.007812 ETA:   0h 0m 0s

Number of examples: 1632
Precision: 0.9938725490196079
Recall: 0.9938725490196079


Progress: 100.0% words/sec/thread: 4188706 lr:  0.000000 avg.loss:  0.007582 ETA:   0h 0m 0s


In [8]:
model = fasttext.train_supervised(input=train_data, epoch=50)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])


Read 1M words
Number of words:  61221
Number of labels: 2
Progress:  93.8% words/sec/thread: 4177617 lr:  0.006244 avg.loss:  0.025701 ETA:   0h 0m 0s

Number of examples: 1632
Precision: 0.9926470588235294
Recall: 0.9926470588235294


Progress: 100.0% words/sec/thread: 3995951 lr:  0.000000 avg.loss:  0.024312 ETA:   0h 0m 0s


In [9]:
from nltk.tokenize import word_tokenize
max_words = 200
human_df['text'] = human_df['text'].apply(preprocess_text)
human_df['text'] = human_df['text'].apply(lambda x: ' '.join(word_tokenize(x)[:max_words]))


In [10]:
data = []
for index, row in human_df.iterrows():
    label = '__label__' + str(row['label'])
    text = row['text']
    example = label + ' ' + text
    data.append(example)

# Save the examples to a text file
with open('SILINECEK.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')

In [11]:
result = model.test('SILINECEK.txt')


In [12]:
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])

Number of examples: 4000
Precision: 0.998
Recall: 0.998
